In [1]:
import torch
from collections import OrderedDict

In [4]:
from typing import Any

import torch
import torch.nn as nn

def isExists(path):
    return path is not None and Path(path).is_file()

class Base(nn.Module):

    def _forward_unimplemented(self, *input: Any) -> None:
        pass

    def __init__(self):
        super(Base, self).__init__()
        self.name = 'Base'
        self._exists = isExists

    def load(self, resume=None, pretrained=None):
        if self._exists(resume):
            print('resume training from:', resume)
            self.load_state_dict(torch.load(resume, map_location=lambda storage, loc: storage))
        elif self._exists(pretrained):
            print('load pretrained model:', pretrained)
            self.load_state_dict(torch.load(pretrained, map_location=lambda storage, loc: storage))
        else:
            print('do not have pretrained model')

    def save(self, path):
        torch.save(self.state_dict(), path)

In [70]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(Base):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(planes),
            nn.ReLU(inplace=True))
        self.conv2 = nn.Sequential(
            nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(planes),
        )

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        residual = x
        x = self.conv1(x)
        x = self.conv2(x)
        return F.relu(x + self.shortcut(residual))


class Bottleneck(Base):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_planes, planes, kernel_size=1, bias=False),
            nn.BatchNorm2d(planes),
            nn.ReLU(inplace=True)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(planes),
            nn.ReLU(inplace=True)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(planes, self.expansion * planes, kernel_size=1, bias=False),
            nn.BatchNorm2d(self.expansion * planes)
        )

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        residual = x
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        return F.relu(x + self.shortcut(residual))


resnet_spec = {18: (BasicBlock, [2, 2, 2, 2]),
               34: (BasicBlock, [3, 4, 6, 3]),
               50: (Bottleneck, [3, 4, 6, 3]),
               101: (Bottleneck, [3, 4, 23, 3]),
               152: (Bottleneck, [3, 8, 36, 3])}


class PoseResNet(Base):

    def __init__(self):
        super(PoseResNet, self).__init__()
        self.name = 'pose-resnet'

        self.in_planes = 64

        self.deconv_with_bias = False
        block, layers = resnet_spec[NUM_LAYERS]

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)

        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0], stride=1)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        # used for deconv layers
        self.deconv_layers = self._make_deconv_layer(
            3,
            [256, 256, 256],
            [4, 4, 4],
        )

        self.final_layer = nn.Conv2d(
            in_channels=256,
            out_channels=16,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True
        )

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion

        return nn.Sequential(*layers)

    def _make_deconv_layer(self, num_layers, num_filters, num_kernels):
        assert num_layers == len(num_filters), 'ERROR: num_deconv_layers is different len(num_deconv_filters)'
        assert num_layers == len(num_kernels), 'ERROR: num_deconv_layers is different len(num_deconv_filters)'
        deconv_cfg = {2: (0, 0), 3: (1, 1), 4: (1, 0)}

        layers = []
        for i in range(num_layers):
            kernel, planes, (padding, output_padding) = num_kernels[i], num_filters[i], deconv_cfg[num_kernels[i]]

            layers.append(
                nn.Sequential(
                    nn.ConvTranspose2d(
                        in_channels=self.in_planes,
                        out_channels=planes,
                        kernel_size=kernel,
                        stride=2,
                        padding=padding,
                        output_padding=output_padding,
                        bias=self.deconv_with_bias),
                    nn.BatchNorm2d(planes),
                    nn.ReLU(inplace=True)
                )
            )
            self.in_planes = planes

        return nn.Sequential(*layers)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.deconv_layers(x)
        x = self.final_layer(x)

        return x

    def load(self, resume=None, pretrained=None):
        if self._exists(resume):
            print('resume training from:', resume)
            self.load_state_dict(torch.load(resume, map_location=lambda storage, loc: storage))
        elif self._exists(pretrained):
            print('load pretrained model:', pretrained)

            # 预训练backbone载入
            model_dict = self.state_dict()
            state_dict = {k: v for k, v in torch.load(pretrained).items() if k in model_dict}
            model_dict.update(state_dict)
            self.load_state_dict(model_dict)

            # 初始化反卷积层
            print('=> init deconv weights from normal distribution')
            for m in self.deconv_layers.modules():
                if isinstance(m, nn.ConvTranspose2d):
                    nn.init.normal_(m.weight, std=0.001)
                    if self.deconv_with_bias:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, nn.BatchNorm2d):
                    nn.init.constant_(m.weight, 1)
                    nn.init.constant_(m.bias, 0)

            # 初始化最终输出层
            print('=> init final conv weights from normal distribution')
            for m in self.final_layer.modules():
                if isinstance(m, nn.Conv2d):
                    nn.init.normal_(m.weight, std=0.001)
                    nn.init.constant_(m.bias, 0)
        else:
            print('do not have pretrained model')
            # 初始化反卷积层
            print('=> init deconv weights from normal distribution')
            for m in self.deconv_layers.modules():
                if isinstance(m, nn.ConvTranspose2d):
                    nn.init.normal_(m.weight, std=0.001)
                    if self.deconv_with_bias:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, nn.BatchNorm2d):
                    nn.init.constant_(m.weight, 1)
                    nn.init.constant_(m.bias, 0)

            # 初始化最终输出层
            print('=> init final conv weights from normal distribution')
            for m in self.final_layer.modules():
                if isinstance(m, nn.Conv2d):
                    nn.init.normal_(m.weight, std=0.001)
                    nn.init.constant_(m.bias, 0)

In [71]:
# root = 'D:/Home/Storehouse/pose_estimation/imagenet-pretrain/resnet50.pth'
# root = '/Users/wucheng/仓库/pose_estimation/imagenet-pretrain/resnet50.pth'
# root = '/Users/wucheng/仓库/pose_estimation/imagenet-pretrain/resnet101.pth'
# root = '/Users/wucheng/仓库/pose_estimation/imagenet-pretrain/resnet152.pth'

# root = '/Users/wucheng/仓库/pose_estimation/trained/baseline/pose_resnet_50_256x256.pth.tar'
root = '/Users/wucheng/仓库/pose_estimation/trained/baseline/pose_resnet_50_256x256.pth.tar'
root = '/Users/wucheng/仓库/pose_estimation/trained/baseline/pose_resnet_50_256x256.pth.tar'

In [72]:
def getDictStatus(d, f=lambda x : True):
    d = list(filter(f, d.keys()))
    print(len(d))
    for k in d:
        print(k)

In [73]:
NUM_LAYERS = 50
model = PoseResNet()
model_dict = model.state_dict()

In [74]:
state_dict = torch.load(root, map_location=torch.device('cpu'))

In [75]:
new_dict = OrderedDict()

In [76]:
getDictStatus(model_dict, lambda x : 'num_batches_tracked' not in x)

282
conv1.weight
bn1.weight
bn1.bias
bn1.running_mean
bn1.running_var
layer1.0.conv1.0.weight
layer1.0.conv1.1.weight
layer1.0.conv1.1.bias
layer1.0.conv1.1.running_mean
layer1.0.conv1.1.running_var
layer1.0.conv2.0.weight
layer1.0.conv2.1.weight
layer1.0.conv2.1.bias
layer1.0.conv2.1.running_mean
layer1.0.conv2.1.running_var
layer1.0.conv3.0.weight
layer1.0.conv3.1.weight
layer1.0.conv3.1.bias
layer1.0.conv3.1.running_mean
layer1.0.conv3.1.running_var
layer1.0.shortcut.0.weight
layer1.0.shortcut.1.weight
layer1.0.shortcut.1.bias
layer1.0.shortcut.1.running_mean
layer1.0.shortcut.1.running_var
layer1.1.conv1.0.weight
layer1.1.conv1.1.weight
layer1.1.conv1.1.bias
layer1.1.conv1.1.running_mean
layer1.1.conv1.1.running_var
layer1.1.conv2.0.weight
layer1.1.conv2.1.weight
layer1.1.conv2.1.bias
layer1.1.conv2.1.running_mean
layer1.1.conv2.1.running_var
layer1.1.conv3.0.weight
layer1.1.conv3.1.weight
layer1.1.conv3.1.bias
layer1.1.conv3.1.running_mean
layer1.1.conv3.1.running_var
layer1.2.con

In [77]:
getDictStatus(state_dict)

282
conv1.weight
bn1.weight
bn1.bias
bn1.running_mean
bn1.running_var
layer1.0.conv1.weight
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.bn1.running_mean
layer1.0.bn1.running_var
layer1.0.conv2.weight
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.0.bn2.running_mean
layer1.0.bn2.running_var
layer1.0.conv3.weight
layer1.0.bn3.weight
layer1.0.bn3.bias
layer1.0.bn3.running_mean
layer1.0.bn3.running_var
layer1.0.downsample.0.weight
layer1.0.downsample.1.weight
layer1.0.downsample.1.bias
layer1.0.downsample.1.running_mean
layer1.0.downsample.1.running_var
layer1.1.conv1.weight
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.bn1.running_mean
layer1.1.bn1.running_var
layer1.1.conv2.weight
layer1.1.bn2.weight
layer1.1.bn2.bias
layer1.1.bn2.running_mean
layer1.1.bn2.running_var
layer1.1.conv3.weight
layer1.1.bn3.weight
layer1.1.bn3.bias
layer1.1.bn3.running_mean
layer1.1.bn3.running_var
layer1.2.conv1.weight
layer1.2.bn1.weight
layer1.2.bn1.bias
layer1.2.bn1.running_mean
layer1.2.bn1.running_var

In [78]:
count = 0
for i in model_dict:
    if 'num_batches_tracked' not in i:
        k = i
        if 'deconv' in i:
            i = i.replace('0.0', '0')
            i = i.replace('0.1', '1')
            i = i.replace('1.0', '3')
            i = i.replace('1.1', '4')
            i = i.replace('2.0', '6')
            i = i.replace('2.1', '7')  
        else:
            i = i.replace('conv1.0', 'conv1')
            i = i.replace('conv1.1', 'bn1')
            i = i.replace('conv2.0', 'conv2')
            i = i.replace('conv2.1', 'bn2')
            i = i.replace('conv3.0', 'conv3')
            i = i.replace('conv3.1', 'bn3')
            i = i.replace('shortcut', 'downsample')
        new_dict[k] = state_dict[i]
        count += 1
print(count)

282


In [79]:
getDictStatus(new_dict)

282
conv1.weight
bn1.weight
bn1.bias
bn1.running_mean
bn1.running_var
layer1.0.conv1.0.weight
layer1.0.conv1.1.weight
layer1.0.conv1.1.bias
layer1.0.conv1.1.running_mean
layer1.0.conv1.1.running_var
layer1.0.conv2.0.weight
layer1.0.conv2.1.weight
layer1.0.conv2.1.bias
layer1.0.conv2.1.running_mean
layer1.0.conv2.1.running_var
layer1.0.conv3.0.weight
layer1.0.conv3.1.weight
layer1.0.conv3.1.bias
layer1.0.conv3.1.running_mean
layer1.0.conv3.1.running_var
layer1.0.shortcut.0.weight
layer1.0.shortcut.1.weight
layer1.0.shortcut.1.bias
layer1.0.shortcut.1.running_mean
layer1.0.shortcut.1.running_var
layer1.1.conv1.0.weight
layer1.1.conv1.1.weight
layer1.1.conv1.1.bias
layer1.1.conv1.1.running_mean
layer1.1.conv1.1.running_var
layer1.1.conv2.0.weight
layer1.1.conv2.1.weight
layer1.1.conv2.1.bias
layer1.1.conv2.1.running_mean
layer1.1.conv2.1.running_var
layer1.1.conv3.0.weight
layer1.1.conv3.1.weight
layer1.1.conv3.1.bias
layer1.1.conv3.1.running_mean
layer1.1.conv3.1.running_var
layer1.2.con

In [80]:
torch.save(new_dict, '/Users/wucheng/仓库/pose_estimation/trained/baseline/pose-resnet50-m.pth')